In [9]:
import pandas as pd
from tableone import TableOne
from src.constants import RELEVANT_DESA_BAD

# Load Epitope Database

In [10]:
path_cohort = '~/UMCUtrecht/KaplanMeier/DESAsurvival_original.csv'
path_antibody = '~/UMCUtrecht/20211104_mismatch_ep_db.pickle'
df_abs = pd.read_pickle(path_antibody)
# There is also another DESA column in the df_cohort.
df_cohort = pd.read_csv(path_cohort, sep=';').drop('DESA', axis=1)
df = df_abs.merge(df_cohort, on='TransplantID')

float_columns =  ['DialysisYears', 'CIPHour_DBD', 'CIPHour_DCD']
for col in float_columns:
    df[col] = df[col].apply(lambda x: x.replace(',', '.')).astype('float')

In [11]:
print(df.columns)
print(df.shape)

Index(['TransplantID', 'Epitope_Mismatch', 'DESA_Status', 'EpvsHLA_Donor',
       '#DESA', 'DESA', 'RecipientAge_NOTR', 'RecipientSex_NOTR',
       'DonorAge_NOTR', 'DonorSex_NOTR',
       ...
       'NumDESA_A', 'NumDESA_B', 'NumDESA_C', 'NumDESA_DRB1', 'NumDESA_DQA1',
       'NumDESA_DQB1', 'NumDESA_DRB345', 'numDESA_ABDRDQ', 'DESA_ABDRDQ',
       'DESA_ABDRDQ_YesNo'],
      dtype='object', length=109)
(4690, 109)


###  Table 1

In [4]:
# selected variables
cols = [
    'DESA_Status', 
    'RecipientAge_NOTR', 'RecipientSex_NOTR', 'DialysisYears',
    'DonorAge_NOTR', 'DonorSex_NOTR', 'TypeOfDonor_NOTR', 'TypeCadaveric_NOTR', 'CIPHour_DBD', 'CIPHour_DCD',
    'Retransplant', 
    # 'InductionTherapy',
    
]
df_t1 = df[cols]
df_t1['DESA_Status'] = df_t1['DESA_Status'].apply(lambda x:'DESA' if x == 'DESA' else 'No DESA')


/var/folders/dz/h73npvns6dscvynm6d4050wc0000gn/T/ipykernel_3706/391310423.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['DESA_Status'] = df_t1['DESA_Status'].apply(lambda x:'DESA' if x == 'DESA' else 'No DESA')


In [6]:
# columns containing categorical variables
categorical = ['RecipientSex_NOTR', 'DonorSex_NOTR', 'TypeOfDonor_NOTR', 'Retransplant', 'TypeCadaveric_NOTR']

# optionally, a categorical variable for stratification
groupby = ['DESA_Status']

# rename the death column
labels={
    'DonorAge_NOTR': 'Donor Age',
    'DonorSex_NOTR': 'Donor Sex',
    'RecipientAge_NOTR': 'Recipiet Age',
    'RecipientSex_NOTR': 'Recipiet Sex',
    'TypeOfDonor_NOTR': 'Donor Type',
    'TypeCadaveric_NOTR': 'Deceased Donor Type',
    'CIPHour_DBD': 'Cold Ischemia Time-DBD',
    'CIPHour_DCD': 'Cold Ischemia Time-DCD',
}

# link to documentation https://github.com/tompollard/tableone/blob/master/tableone/tableone.py
table1 = TableOne(
    df_t1, 
    groupby=groupby,
    missing=False,
    categorical=categorical,
    pval=True,
    rename=labels,
    htest_name=True,
);
table1

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use grou

Grouped by DESA_Status                                                     
                                                                  Overall         DESA      No DESA P-Value               Test
n                                                                    4690          439         4251                           
Recipiet Age, mean (SD)                                       45.4 (14.4)  44.6 (13.7)  45.5 (14.4)   0.168  Two Sample T-test
Recipiet Sex, n (%)               Female                      1879 (40.1)   267 (60.8)  1612 (37.9)  <0.001        Chi-squared
                                  Male                        2811 (59.9)   172 (39.2)  2639 (62.1)                           
DialysisYears, mean (SD)                                        2.8 (2.5)    3.3 (3.0)    2.7 (2.4)  <0.001  Two Sample T-test
Donor Age, mean (SD)                                          44.3 (15.0)  45.1 (15.3)  44.2 (15.0)   0.229  Two Sample T-test
Donor Sex, n (%)                  Female                      2366 (50.4)   193 (44.0)  2173 (51.1)   0.005        Chi-squared
                                  Male                        2324 (49.6)   246 (56.0)  2078 (48.9)                           
Donor Type, n (%)                 Deceased                    3235 (69.0)   315 (71.8)  2920 (68.7)   0.205        Chi-squared
                                  Living                      1455 (31.0)   124 (28.2)  1331 (31.3)                           
Deceased Donor Type, n (%)        Heartbeating                2427 (75.0)   248 (78.7)  2179 (74.6)   0.126        Chi-squared
                                  Non-heartbeating             808 (25.0)    67 (21.3)   741 (25.4)                           
Cold Ischemia Time-DBD, mean (SD)                             11.3 (12.2)  12.8 (12.5)  11.2 (12.1)   0.007  Two Sample T-test
Cold Ischemia Time-DCD, mean (SD)                               3.8 (8.7)    3.3 (8.3)    3.8 (8.7)   0.262  Two Sample T-test
Retransplant, n (%)               No                          3970 (84.6)   239 (54.4)  3731 (87.8)  <0.001        Chi-squared
                                  Yes                          720 (15.4)   200 (45.6)   520 (12.2)

## Table 2 variants

In [12]:
cols = [
    'DESA_Status', 
    'RecipientAge_NOTR', 'RecipientSex_NOTR', 'DialysisYears',
    'DonorAge_NOTR', 'DonorSex_NOTR', 'TypeOfDonor_NOTR', 'TypeCadaveric_NOTR', 'CIPHour_DBD', 'CIPHour_DCD',
    'Retransplant', 'DESA'
]

df_desa = df[cols][df.DESA_Status == 'DESA']
df_desa['DESA_Type'] = df_desa['DESA'].apply(lambda x: 'Bad DESA' if x & RELEVANT_DESA_BAD else 'Other DESA')
df_desa = df_desa.drop(['DESA_Status', 'DESA'], axis=1)


In [13]:
# columns containing categorical variables
categorical = ['RecipientSex_NOTR', 'DonorSex_NOTR', 'TypeOfDonor_NOTR', 'Retransplant', 'TypeCadaveric_NOTR']

# optionally, a categorical variable for stratification
groupby = ['DESA_Type']

# rename the death column
labels={
    'DonorAge_NOTR': 'Donor Age',
    'DonorSex_NOTR': 'Donor Sex',
    'RecipientAge_NOTR': 'Recipiet Age',
    'RecipientSex_NOTR': 'Recipiet Sex',
    'TypeOfDonor_NOTR': 'Donor Type',
    'TypeCadaveric_NOTR': 'Deceased Donor Type',
    'CIPHour_DBD': 'Cold Ischemia Time-DBD',
    'CIPHour_DCD': 'Cold Ischemia Time-DCD',
}

# link to documentation https://github.com/tompollard/tableone/blob/master/tableone/tableone.py
table2 = TableOne(
    df_desa, 
    groupby=groupby,
    missing=False,
    categorical=categorical,
    pval=True,
    rename=labels,
    htest_name=True,
);
table2

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use grou

Grouped by DESA_Type                                                     
                                                                Overall     Bad DESA   Other DESA P-Value               Test
n                                                                   439          196          243                           
Recipiet Age, mean (SD)                                     44.6 (13.7)  44.4 (13.6)  44.7 (13.8)   0.834  Two Sample T-test
Recipiet Sex, n (%)               Female                     267 (60.8)   121 (61.7)   146 (60.1)   0.799        Chi-squared
                                  Male                       172 (39.2)    75 (38.3)    97 (39.9)                           
DialysisYears, mean (SD)                                      3.3 (3.0)    3.3 (2.7)    3.3 (3.3)   0.751  Two Sample T-test
Donor Age, mean (SD)                                        45.1 (15.3)  46.3 (15.1)  44.2 (15.5)   0.164  Two Sample T-test
Donor Sex, n (%)                  Female                     193 (44.0)    92 (46.9)   101 (41.6)   0.302        Chi-squared
                                  Male                       246 (56.0)   104 (53.1)   142 (58.4)                           
Donor Type, n (%)                 Deceased                   315 (71.8)   148 (75.5)   167 (68.7)   0.143        Chi-squared
                                  Living                     124 (28.2)    48 (24.5)    76 (31.3)                           
Deceased Donor Type, n (%)        Heartbeating               248 (78.7)   113 (76.4)   135 (80.8)   0.405        Chi-squared
                                  Non-heartbeating            67 (21.3)    35 (23.6)    32 (19.2)                           
Cold Ischemia Time-DBD, mean (SD)                           12.8 (12.5)  13.4 (12.6)  12.4 (12.4)   0.403  Two Sample T-test
Cold Ischemia Time-DCD, mean (SD)                             3.3 (8.3)    3.9 (8.5)    2.9 (8.1)   0.227  Two Sample T-test
Retransplant, n (%)               No                         239 (54.4)   103 (52.6)   136 (56.0)   0.537        Chi-squared
                                  Yes                        200 (45.6)    93 (47.4)   107 (44.0)